# Verbatim compilation

Usually, when you run a circuit on a QPU, behind the scenes, Amazon Braket will do a series of compilation steps to optimize your circuit and map the abstract circuit to the physical qubits on the QPU. However, in many situations,  such as for error mitigation or benchmarking experiments, researchers require full control of the qubits and the gates that are being applied. In a [previous notebook](https://github.com/aws/amazon-braket-examples/blob/main/examples/braket_features/Allocating_Qubits_on_QPU_Devices.ipynb), we showed you how to manually allocate the qubits of your circuit, i.e., how you can exactly define which logical qubit maps to which physical qubit. In this notebook, you will learn how to use _verbatim compilation_ to run your circuits exactly as defined without any modification during the compilation process.

### Table of contents:

* [Recap: Running circuits on Amazon Braket](#Recap)
* [Using verbatim compilation to run circuits without further compilation](#Verbatim)
* [Programming verbatim circuits onto the Rigetti device](#Rigetti)
* [Programming verbatim circuits onto the Oxford Quantum Circuits device](#OQC)

## Recap: Running circuits on Amazon Braket  <a class="anchor" id="Recap"></a>

Let us begin with importing the usual dependencies. Verbatim compilation is supported by all Rigetti devices, and we will use the Aspen-M-1 device for this demonstration.

In [1]:
# general imports
import boto3
from braket.aws import AwsDevice
from braket.circuits import Circuit
from math import pi
import networkx as nx

When you run a circuit on Amazon Braket, different compilation and optimization steps occur before the circuit is executed on the selected QPU. First, the gates of your circuit are decomposed into the _native gates_ of the QPU. Let's first remember what the native gates of the Aspen-M-1 device are:

In [2]:
# set up the Rigetti Aspen-M-1 device
device = AwsDevice("arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-1")

# list the native gate set
print("The native gates for the", device.name, "device are:")
for gate in device.properties.paradigm.nativeGateSet:
    print(gate)

The native gates for the Aspen-M-1 device are:
rx
rz
cz
cphaseshift
xy


If we create a circuit with gates that are not part of that list, the gates will automatically be decomposed into a gate set that can be executed on the device.

<div class="alert alert-block alert-info">
    <b>Note:</b> This notebook uses the Rigetti Aspen-M-1 device. When you run this notebook, make sure the device is currently available. You can find QPU availability windows on the <a href="https://us-west-1.console.aws.amazon.com/braket/home?region=us-west-1#/devices">Devices page</a> in the Amazon Braket Console
</div>

In [3]:
bell = Circuit().h(100).cnot(100,101)
print(bell)
result = device.run(bell, shots=1000).result()
print(result.measurement_counts)

T    : |0|1|
            
q100 : -H-C-
          | 
q101 : ---X-

T    : |0|1|
Counter({'11': 516, '00': 423, '01': 33, '10': 28})


Let's have a look at the circuit that was actually executed. 

In [4]:
meta = result.additional_metadata.rigettiMetadata
print(meta.compiledProgram)

DECLARE ro BIT[2]
PRAGMA INITIAL_REWIRING "PARTIAL"
RESET
RZ(pi/2) 15
RX(pi/2) 15
RZ(-pi/2) 16
RX(pi/2) 16
CZ 16 15
RZ(-pi/2) 15
RX(-pi/2) 16
RZ(pi/2) 16
MEASURE 16 ro[1]
MEASURE 15 ro[0]



As you can see, the original gates, `h` and `cnot`, were decomposed into the native `rz`, `rx`, `cz`, and `cphase` gates. You will also note that the abstract qubit indices 100 and 101 (which were chosen as an example) were remapped to qubits that actually exist on the device, in this case, 21 and 36, respectively. At the time when you run this notebook, you might see a different remapping, as the compiler takes into account the latest calibration of the device and tries to map to the qubits that yield the best results. 

The compiler further performs circuit optimizations to minimize the number of operations, e.g., by removing redundant gates. Let's have a look at a single-qubit circuit containing two subsequent `x` gates:

In [5]:
identity = Circuit().x(0).x(0)

Of course, if you do the math, two consecutive `x` gates just cancel each other out, and we are left with an empty circuit after compilation.

In [6]:
result = device.run(identity, shots=1000).result()
compiled_program = result.additional_metadata.rigettiMetadata.compiledProgram
print(compiled_program)
print(result.measurement_counts)

DECLARE ro BIT[1]
PRAGMA INITIAL_REWIRING "PARTIAL"
RESET
MEASURE 141 ro[0]

Counter({'0': 994, '1': 6})


## Using verbatim compilation to run circuits without further compilation <a class="anchor" id="Verbatim"></a>

In some cases, however, you may need to run circuits exactly as defined, without any further modifications by the compiler. For instance, if you want to benchmark device performance you may want to control exactly which gates are executed on the hardware. Similarly, [certain error mitigation protocols](https://arxiv.org/pdf/2005.10921.pdf) require insertion of additional, redundant operations that would normally be removed by the compiler but are essential for the protocol to work. To prevent circuits (or parts of circuits) from further compiler optimizations, you can use the `add_verbatim_box` function in the Amazon Braket SDK.

<div class="alert alert-block alert-info">
    <b>Note:</b> <code>add_verbatim_box</code> is currently only supported on Rigetti devices.
</div>

When you define a (sub-)circuit in a verbatim box, you need to make sure that everything inside the verbatim box can be executed on the device _exactly_ as you defined it. This means that
1. The circuit can only use qubit indices that exist on the device
2. All gates of the circuit have to be part of the native gate set of the device
3. All multi-qubit gates have to be between qubits that are connected according to the connectivity graph of the device 

### Getting started with a minimal example
Let's have a look at the simple example from before of an identify circuit. 

In [7]:
identity = Circuit().rx(0,pi).rx(0,pi)

Here, we have chosen the Rigetti-native `rx` gate, which is identical to `x` if we choose the angle to be $\pi$. Next, we wrap this circuit in a verbatim box to prevent the compiler from collapsing the two gates. You can see the circuit diagram indicates the start and end of the verbatim box.

In [8]:
circ = Circuit().add_verbatim_box(identity)
print(circ)

T  : |      0      |   1    |   2    |     3     |
                                                  
q0 : -StartVerbatim-Rx(3.14)-Rx(3.14)-EndVerbatim-

T  : |      0      |   1    |   2    |     3     |


When we run the circuit, both gates will be executed on the device.

<div class="alert alert-block alert-info">
    <b>Note:</b> To run a circuit that contains a verbatim box the <code>disable_qubit_rewiring</code> flag must be set to <code>True</code> 
</div>

In [ ]:
# set up the Rigetti Aspen-M-1 device
rigetti = AwsDevice("arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-1")

result = rigetti.run(circ, shots=1000, disable_qubit_rewiring=True).result()
compiled_program = result.additional_metadata.rigettiMetadata.compiledProgram
print(compiled_program)
print(result.measurement_counts)

## Programming verbatim circuits onto the Rigetti device  <a class="anchor" id="Rigetti"></a>

As we have mentioned above, to build circuits with multi-qubit gates, you need to take into consideration the connectivity graph of the device. When a circuit contains a verbatim box, automatic qubit rewiring has to be disables, and you have manually allocate the qubits on the device that you want to use for your circuit.
You can access the connectivity graph on the [device detail page](https://console.aws.amazon.com/braket/home?region=us-west-1#/devices/arn:aws:braket:us-west-2::device/qpu/rigetti/Aspen-M-1) in the Amazon Braket Console, or by using the code below.

In [ ]:
# access and visualize the device topology
# note that device topology can change day-to-day based on edge fidelity data
print(rigetti.properties.paradigm.connectivity.connectivityGraph)
nx.draw_kamada_kawai(rigetti.topology_graph, with_labels=True, font_color="white")

From the connectivity graph, you can see that qubits 11, 10, and 17 are connected in a line, and with the code in the next cell you can access their respective 2-qubit gate fidelities to make sure you have selected a high-quality qubit subset. 

In [ ]:
print(rigetti.properties.provider.specs["2Q"]["10-11"])
print(rigetti.properties.provider.specs["2Q"]["10-17"])

<div class="alert alert-block alert-info">
    <b>Note:</b> At the time when you run this notebook the fidelity numbers may be different as QPU devices are periodically recalibrated
</div>

After selecting the qubits and validating their gate fidelities, you can now construct a circuit and run it. 

In [ ]:
circ = Circuit().xy(10,11,pi/4).xy(10,17,pi/2).rx(10,pi).xy(10,11,pi/4)
verbatim_circ = Circuit().add_verbatim_box(circ)
print(verbatim_circ)

In [ ]:
result = rigetti.run(verbatim_circ, shots=1000, disable_qubit_rewiring=True).result()
compiled_program = result.additional_metadata.rigettiMetadata.compiledProgram
print(compiled_program)
print(result.measurement_counts)

As expected, the program is faithfully executed exactly as it was defined.

### Defining verbatim subcircuits 

In some situations, you might only be interested in executing parts of your circuits verbatim, and have the compiler nativize and optimize the rest. The next example demonstrates how to do this. Let's get started by defining the subcircuit we want to execute verbatim.

In [ ]:
verbatim_subcirc = Circuit().rx(10,pi).rx(10,pi)

Next, define the part of the circuit you want to compiler to process, so you don't have to worry about nativizing gates. 

In [ ]:
subcirc1 = Circuit().cnot(10,11).cnot(10,17)
subcirc2 = Circuit().cnot(10,17).cnot(10,11)

Finally, put everything together

In [ ]:
circ = subcirc1.add_verbatim_box(verbatim_subcirc).add_circuit(subcirc2)
print(circ)

In [ ]:
result = rigetti.run(circ, shots=1000, disable_qubit_rewiring=True).result()
compiled_program = result.additional_metadata.rigettiMetadata.compiledProgram
print(compiled_program)
print(result.measurement_counts)

Inspecting the compiled program, you can see that the `cnot` gates were nativized, however, the two `rx` gates in the middle of the circuit remain unaltered, and were not removed by the compiler. 

## Programming verbatim circuits onto the Oxford Quantum Circuits device <a class="anchor" id="OQC"></a>

Oxford Quantum Circuits (OQC) also supports verbatim compilation. As mentioned above, a verbatim circuit does not allow automatic qubit rewiring. All qubits have to be allocated using the indices that exist on the device. The gates of a verbatim circuit have to come from the native gate set, and multi-qubits gates can only be applied according to the connectivity graph of the device. First, let's look at OQC's native gate set.

In [ ]:
# set up the OQC Lucy device
oqc_device = AwsDevice("arn:aws:braket:eu-west-2::device/qpu/oqc/Lucy")

# list the native gate set
print("The native gates for the", oqc_device.name, "device are:")
for gate in oqc_device.properties.paradigm.nativeGateSet:
    print(gate)

Let's check out the device connectivity graph to see what qubits are on the device and how they are connected. Notice that connections in the OQC device are unidirectional. This mean that two-qubits gates can only be applied one direction with verbatim compilation. In the topology graph, each arrow represents the direction that a two-qubit gate can be applied in verbatim mode. The tail of the arrow is the first qubit and the head is the second qubit. For example, `ecr(4,3)` is a valid gate while `ecr(3,4)` is not.

In [ ]:
# access and visualize the device topology
print(oqc_device.properties.paradigm.connectivity.connectivityGraph)
nx.draw_kamada_kawai(oqc_device.topology_graph, with_labels=True, font_color="white", arrows=True, arrowsize=30)

With this information, we can create a circuit and run it with verbatim mode. OQC currently only supports verbatim mode for the whole circuit, i.e. the verbatim box has to contain all the gates in a circuit when submitting to the device.

In [ ]:
circ = Circuit().rz(0,pi/4).rz(0,-pi/4).ecr(1,2).x(2).ecr(2,3)
verbatim_circ = Circuit().add_verbatim_box(circ)
print(verbatim_circ)

In [ ]:
result = oqc_device.run(verbatim_circ, shots=1000, disable_qubit_rewiring=True).result()
compiledProgram = result.additional_metadata.oqcMetadata.compiledProgram
print(compiledProgram)
print(result.measurement_counts)

## Conclusion 
This notebook introduced the basic functionality of verbatim compilation of Amazon Braket, that allows you to run circuits or subcircuits to be executed exactly as defined without any compiler modifications. You can find further information in the [Amazon Braket documentation](https://docs.aws.amazon.com/braket/). 